In [1]:
!git clone https://github.com/Auto-annotation-of-Pathology-Images/AAPI_code.git

Cloning into 'AAPI_code'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 24 (delta 4), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [ ]:
%%capture
!apt update && apt install -y openslide-tools
!pip install openslide-python
import openslide

In [5]:
from AAPI_code.Collage_generator.collage_generator import collage_generator
from AAPI_code.Collage_generator.utils import *

# Importing images from CVAT result

In [ ]:
%%capture
!unzip "/content/task_glom-2020_10_03_00_54_30-segmentation mask 1.1.zip"

In [ ]:
# this method only read everything together
segmentation = os.listdir("SegmentationObject")
extracted = [crop_image(extract_image(i)) for i in segmentation]

# Importing cropped images

In [ ]:
segmentation = os.listdir("Cropped_image")
extracted = [np.array(Img.open(i).convert("RGB")) for i in segmentation]

# collage_generator

## Initialization:  
example_image.jpg should be okay for background setting  
you can define another `keras.preprocessing.image.ImageDataGenerator `, when it's None it will use the original setting.

In [6]:
col_gen = collage_generator(label_list= ["glomerulus", "artery"], 
                            canvas_size=(1024,1024),
                            example_image = "/content/AAPI_code/Collage_generator/example_background.jpg",
                            gaussian_noise_constant = 100,
                            patience = 10,
                            imagedatagenerator = None)

## Add Image
`img` can be path, np.ndarray, or PIL image object.  
Personally suggest that set `pre_determined_size = False`, it can help setting a correct "original size" on the canvas, (80,75) is by estimation from the online svs file

In [ ]:
for image in extracted:
    col_gen.add_image(img = image, label = "glomerulus", size = (80,75), pre_determined_size = True)
col_gen.add_image(img = "/content/Image_215_artery_cv2.png", label = "artery", size = (80,75), pre_determined_size = True)

## Generate

Just like that... `ratio_list` defines the ratio of different entity (glomerulus, artery, etc.) you can call `col_gen.label_list` for the actual order, it's okay to have them not sum to 1.

In [ ]:
collage, mask, dictionary = col_gen.generate(item_num = 200, ratio_list = [0.5,0.5])

f, axarr = plt.subplots(1,2)
axarr[0].set_axis_off()
im1 = axarr[0].imshow(collage)
axarr[1].set_axis_off()
im2 = axarr[1].imshow(mask)
plt.colorbar(im2, ax=axarr[1])
print(dictionary)